# Visualization

In [ ]:
import numpy as np
import os
import sys
import time
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, pc_plot_trace

%load_ext autoreload
%autoreload 2

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_60_samples_loop_closure', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PCs)

for i in range(num_scans):
    PCs[i] = NED_to_ENU(PCs[i])

Manual 

In [ ]:
P = PCs[10]
scan = pc_to_scan(P)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

# Plot point cloud
fig.add_trace(pc_plot_trace(P), row=1, col=1)

# Plot scan
for t in scan.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

In [ ]:
scan.planes[2].normal

Iterative visualization (attempt)

In [ ]:
for i in range(num_scans):
    key = input("Waiting for keypress")
    if key == 'q':
        break

    scan = pc_to_scan(PCs[i])

    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

    # Plot new scan
    for t in scan.plot_trace():
        fig.add_trace(t, row=1, col=1)

    # Reduce
    scan.reduce()

    for t in scan.plot_trace():
        fig.add_trace(t, row=1, col=2)

    fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
    fig.show()

In [ ]:
k = input()
print(k)

Open3D

In [ ]:
import open3d as o3d

In [ ]:
verts = scan.planes[0].vertices

In [ ]:
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(verts)
line_set.lines = o3d.utility.Vector2iVector([[0,1],[1,2],[2,3],[3,0]])
o3d.visualization.draw_geometries([line_set])

In [ ]:
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector([[0,1,2],[0,2,3]])
o3d.visualization.draw_geometries([line_set, mesh])

In [ ]:
geoms = scan.o3d_geometries()
o3d.visualization.draw_geometries(geoms)